In [3]:
import time
import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import torch
import cv2
import torch.nn.functional as F
#from model import LeNet5
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#LeNet5模型
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 6, 5, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride =2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride =2))
        self.fc1 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU())
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #nn.Linear()輸入/輸出皆為一維的值, 因此需要將多維的tensor轉成一維
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

if __name__ == '__main__':
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])
    
    trainSet = torchvision.datasets.CIFAR10(root='./Data', train=True, download=True, transform=transform)

    #將訓練集的50000張圖片劃分為12500份, 每份4張圖, 用於mini-batch輸入, shuffle=True表示不同批次的數據使用非固定順序載入
    trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=4, shuffle=False)

    #10種分類集
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    net = LeNet5().to(device)
    criterion = nn.CrossEntropyLoss()   #Loss Function

    #使用SGD(隨機梯度下降), 學習率及動量
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    logfile = open('Log/Training.log', 'w')
    log_timeflag = time.time()
    logfile.write('Train_time: start' + str(log_timeflag) + '\n')
    for epoch in range(35):
        running_loss = 0.0
        printText = ''
        for i, data in enumerate(trainLoader, 0):   #enumerate(sequence, [start=0])
            inputs, labels = data                   #data struct = [4(BatchSize)x3x32x32]
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs, labels = Variable(inputs), Variable(labels) #轉換inputs, 從tensor轉為variable, variable才具有梯度g
            optimizer.zero_grad()                   #初始化Gradient為0

            #forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)   #使用CrossEntropy方式計算Loss值
            loss.backward()                     #反向傳播求梯度
            optimizer.step()                    #使用SGD更新參數

            #每2000批數據print avg Loss value
            running_loss += loss.item()
            if i % 2000 == 1999:
                printText = '[epoch:%d, idx:%5d] loss: %.3f' % (epoch+1, i+1, running_loss/2000)
                print(printText)
                logfile.write(printText+ '\n')
                running_loss = 0.0 
print('Finished Training')
log_timeflag = time.time()
logfile.write('Finished Training: at' + str(log_timeflag))
logfile.close()

#測試集, 將cifar--batches-py 10000張圖片作為測試數據
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=4, shuffle=False)
correct = 0
total = 0
logfile = open('Log/Test.log', 'w')
log_timeflag = time.time()
logfile.write('Test_time: start' + str(log_timeflag) + '\n')
with torch.no_grad():
    for data in testLoader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(Variable(images))
        value, predicted = torch.max(outputs.data, 1)   #outputs.data size = [4x10]
        total += labels.size(0)
        correct += (predicted == labels).sum()
print('Accuracy: %d %%' % (100*correct/total))
logfile.write('Accuracy: ' + str(100*correct/total) + ' %\n')

#Output 10 Classes Accuracy
class_correct = list(0. for i in range (10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testLoader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100*class_correct[i]/class_total[i]))
    logfile.write('Accuracy of ' + str(classes[i]) + ' : ' + str(100*class_correct[i]/class_total[i]) + ' %\n')
logfile.close()
torch.save(net, './Model.pth')

Files already downloaded and verified
[epoch:1, idx: 2000] loss: 2.177
[epoch:1, idx: 4000] loss: 1.856
[epoch:1, idx: 6000] loss: 1.694
[epoch:1, idx: 8000] loss: 1.580
[epoch:1, idx:10000] loss: 1.551
[epoch:1, idx:12000] loss: 1.476
[epoch:2, idx: 2000] loss: 1.416
[epoch:2, idx: 4000] loss: 1.406
[epoch:2, idx: 6000] loss: 1.356
[epoch:2, idx: 8000] loss: 1.299
[epoch:2, idx:10000] loss: 1.329
[epoch:2, idx:12000] loss: 1.282
[epoch:3, idx: 2000] loss: 1.247
[epoch:3, idx: 4000] loss: 1.254
[epoch:3, idx: 6000] loss: 1.219
[epoch:3, idx: 8000] loss: 1.174
[epoch:3, idx:10000] loss: 1.203
[epoch:3, idx:12000] loss: 1.172
[epoch:4, idx: 2000] loss: 1.141
[epoch:4, idx: 4000] loss: 1.160
[epoch:4, idx: 6000] loss: 1.130
[epoch:4, idx: 8000] loss: 1.092
[epoch:4, idx:10000] loss: 1.113
[epoch:4, idx:12000] loss: 1.088
[epoch:5, idx: 2000] loss: 1.059
[epoch:5, idx: 4000] loss: 1.089
[epoch:5, idx: 6000] loss: 1.055
[epoch:5, idx: 8000] loss: 1.018
[epoch:5, idx:10000] loss: 1.039
[epoc

In [6]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.load('./Model.pth')
    model = model.to(device)
    model.eval()

    img = cv2.imread('./TestPic/Bird.jpg')                          # 讀圖
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)   # Resize成Cifar 32x32
    cv2.imshow('Resize Pic', img)
    cv2.imwrite('./TestPic/Resize_Bird.jpg', img)

    trans = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])
    img = trans(img)
    img = img.to(device)
    img = img.unsqueeze(0)      #增維, 配合4維Model, 成為[1,1,28,28]
    output = model(img)
    prob = F.softmax(output, dim=1) #10 classes Percentage
    print(prob)
    value, predicted = torch.max(output.data, 1)
    print(value)
    pred_class = classes[predicted.item()]
    print(pred_class)

tensor([[1.5522e-03, 2.0346e-08, 9.9823e-01, 9.9067e-07, 1.8949e-04, 1.0315e-08,
         9.2958e-06, 2.7830e-08, 1.6225e-05, 1.6152e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
tensor([12.6261], device='cuda:0')
bird
